In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("./processed_data.csv")
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId,sub_comments,is_valid_reconstruction
0,gp:AOqpTOHNHm4OfbjkxEXXa51JwZEHAaDlvfSgN0OU256...,Lex S,https://lh3.googleusercontent.com/-FgDpDeEJLAw...,"I love this app, but I do have one major gripe...",1,3,5.2.0.23,2020-08-05 16:22:04,Any.do is not only a product but also a servic...,2020-08-05 08:02:08,most_relevant,com.anydo,"[{""sub_comment"": ""I love this app, but I do ha...",True
1,gp:AOqpTOEujjLj56XVqumAkipImEqIAU3qTIuQjENPaOK...,Sam van Dijk,https://lh3.googleusercontent.com/-pO3wTIb4myA...,"Trash. Yes, it has some nice nifty features bu...",1,25,5.2.0.23,2020-07-21 22:17:25,"Premium users can edit, create and delete tags...",2020-07-23 15:57:51,most_relevant,com.anydo,"[{""sub_comment"": ""Trash. Yes, it has some nice...",False
2,gp:AOqpTOElISilniODwd6UBrqFngzTtDHLF-G0VLpR2_y...,Hugo Bounoua,https://lh3.googleusercontent.com/a-/AOh14GgxG...,"OMG the UI is awful, seriously you have popup ...",1,8,5.2.0.23,2020-07-22 07:23:35,The Premium ad only shows up when first openin...,2020-07-23 16:20:43,most_relevant,com.anydo,"[{""sub_comment"": ""OMG the UI is awful, serious...",True
3,gp:AOqpTOEkZ75JR5CzVhxoxWa0XVmPanw_pEl1srcJ7yv...,Aishwarya Mishra,https://lh3.googleusercontent.com/a-/AOh14Ghhq...,I've been using the app for a while and since ...,1,20,5.2.0.23,2020-07-19 06:49:15,"Hi, due to new restrictions from Google, the p...",2020-07-22 14:05:56,most_relevant,com.anydo,"[{""sub_comment"": ""I've been using the app for ...",True
4,gp:AOqpTOEtpLcODD_NZOBqR1N7DBbaLdw3Gyz3v3xZAp1...,Mad Scientist,https://lh3.googleusercontent.com/-kIZF4kMt6yY...,"Unable to register with an email. Clicking""con...",1,77,5.2.0.9,2020-07-10 17:59:22,We are unaware of any issues with signing in t...,2020-07-12 08:02:19,most_relevant,com.anydo,"[{""sub_comment"": ""Unable to register with an e...",True


In [4]:
df.loc[1, 'sub_comments']

'[{"sub_comment": "Trash. Yes, it has some nice nifty features but it lacks in complete necessities to actually plan your day.", "topic": "function", "sentiment": -1.0}, {"sub_comment": "You can make a task, perfectly include subtasks and then add a nice color to the task. But no yellow, because that\'s already the fixed color of the \'priority label\'.", "topic": "design", "sentiment": -0.2}, {"sub_comment": "Reminders are great, except you can only have one. If I want to have a reminder on Wednesday 10AM and a reminder on Friday 6PM, I\'d need a different app. Because this app only allows you to set 1 reminder. Frustrating. 0/10", "topic": "function", "sentiment": -1.0}]'

In [5]:
df.loc[1, 'sub_comments']

'[{"sub_comment": "Trash. Yes, it has some nice nifty features but it lacks in complete necessities to actually plan your day.", "topic": "function", "sentiment": -1.0}, {"sub_comment": "You can make a task, perfectly include subtasks and then add a nice color to the task. But no yellow, because that\'s already the fixed color of the \'priority label\'.", "topic": "design", "sentiment": -0.2}, {"sub_comment": "Reminders are great, except you can only have one. If I want to have a reminder on Wednesday 10AM and a reminder on Friday 6PM, I\'d need a different app. Because this app only allows you to set 1 reminder. Frustrating. 0/10", "topic": "function", "sentiment": -1.0}]'

In [6]:
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import ast

# --- 1. Data Loading and Preprocessing ---

def load_and_preprocess(df, text_column: str = 'sub_comments') -> pd.DataFrame:
    """Loads data, extracts sub-comments, and adds necessary columns (Pandas version)."""
    # df = pd.read_json(file_path)

    # Function to safely parse the string representation of the list of dictionaries
    def parse_sub_comments(text: str) -> list:
        try:
            return ast.literal_eval(text)
        except (SyntaxError, ValueError):
            return []  # Return empty list if parsing fails.

    # Apply the parsing function and explode
    df['sub_comments_list'] = df[text_column].apply(parse_sub_comments)
    df = df.explode("sub_comments_list").reset_index(drop=True)

    # Extract sub-comment, topic, and sentiment, handling potential errors
    df['sub_comment'] = df['sub_comments_list'].apply(lambda x: x.get('sub_comment', '') if isinstance(x, dict) else '')
    df['topic'] = df['sub_comments_list'].apply(lambda x: x.get('topic', None) if isinstance(x, dict) else None)
    df['sentiment'] = df['sub_comments_list'].apply(lambda x: x.get('sentiment', None) if isinstance(x, dict) else None)

    # Drop unnecessary columns and rows with missing values
    df = df.drop(columns=[text_column, 'sub_comments_list'])
    df = df.dropna()

    return df[['sub_comment', 'topic', 'sentiment']]

In [7]:
clean = load_and_preprocess(df)
clean.head()

,sub_comment,topic,sentiment
0,"I love this app, but I do have one major gripe...",price,-1.0
1,"Trash. Yes, it has some nice nifty features bu...",function,-1.0
2,"You can make a task, perfectly include subtask...",design,-0.2
3,"Reminders are great, except you can only have ...",function,-1.0
4,"OMG the UI is awful, seriously you have popup ...",design,-1.0


In [8]:
import re

In [9]:
import re

def get_unique_words(df: pd.DataFrame, text_column: str) -> list:
    """
    Extracts a list of unique words from a specified text column in a Pandas DataFrame.

    Args:
        df: The Pandas DataFrame.
        text_column: The name of the column containing the text.

    Returns:
        A list of unique words (lowercase, without punctuation), sorted alphabetically.
        Returns an empty list if the column is not found or if there's an error.
    """
    if text_column not in df.columns:
        print(f"Error: Text column '{text_column}' not found in DataFrame.")
        return []

    # Drop rows with missing values in the text column.
    df = df.dropna(subset=[text_column])

    # 1. Combine all text into a single string (efficient).
    all_text = ' '.join(df[text_column].astype(str).tolist()) #astype to string

    # 2. Lowercase and remove punctuation (using regular expressions for efficiency).
    all_text = all_text.lower()
    all_text = re.sub(r'[^\w\s]', '', all_text)  # Remove punctuation

    # 3. Split into words and get unique words using a set.
    words = all_text.split()
    unique_words = set(words)

    # 4. Convert back to a list and sort (for consistent results).
    unique_words_list = sorted(list(unique_words))

    return unique_words_list

In [10]:
word_list = get_unique_words(clean, 'sub_comment')

In [11]:
len(word_list)

8425

In [18]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import ast
import numpy as np
from sklearn.metrics import mean_squared_error, accuracy_score

# --- Data Loading and Preprocessing (same as before) ---
def load_and_preprocess(file_path: str, text_column: str = 'review_text') -> pd.DataFrame:
    """Loads data, extracts sub-comments, adds necessary columns (Pandas)."""
    try:
        df = pd.read_json(file_path)
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

    def parse_sub_comments(text: str) -> list:
        try:
            return ast.literal_eval(text)
        except (SyntaxError, ValueError):
            return []

    df['sub_comments_list'] = df[text_column].apply(parse_sub_comments)
    df = df.explode("sub_comments_list").reset_index(drop=True)

    df['sub_comment'] = df['sub_comments_list'].apply(lambda x: x.get('sub_comment', '') if isinstance(x, dict) else '')
    df['topic'] = df['sub_comments_list'].apply(lambda x: x.get('topic', None) if isinstance(x, dict) else None)
    df['sentiment'] = df['sub_comments_list'].apply(lambda x: x.get('sentiment', None) if isinstance(x, dict) else None)

    df = df.drop(columns=[text_column, 'sub_comments_list'])
    df = df.dropna()
    return df

# --- Custom Dataset (same as before) ---
class SubCommentDatasetMultiTask(Dataset):
    def __init__(self, dataframe: pd.DataFrame, tokenizer, max_length: int):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.unique_topics = sorted(self.dataframe['topic'].unique())
        self.topic_to_id = {topic: i for i, topic in enumerate(self.unique_topics)}
        self.num_labels = len(self.unique_topics)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row['sub_comment']
        topic = row['topic']
        sentiment = row['sentiment']
        encoding = self.tokenizer(text, add_special_tokens=True, max_length=self.max_length,
            padding='max_length', truncation=True, return_attention_mask=True, return_tensors='pt')
        topic_label = self.topic_to_id[topic]
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten() if 'token_type_ids' in encoding else torch.zeros_like(encoding['input_ids']).flatten(),
            'topic_label': torch.tensor(topic_label, dtype=torch.long),
            'sentiment_score': torch.tensor(sentiment, dtype=torch.float32),
            'text': text
        }

# --- Multi-Task Model (same as before) ---
class BertMultiTask(nn.Module):
    def __init__(self, base_model_name: str, num_topics: int):
        super().__init__()
        self.bert = BertModel.from_pretrained(base_model_name)
        self.dropout = nn.Dropout(0.1)
        self.topic_classifier = nn.Linear(self.bert.config.hidden_size, num_topics)
        self.sentiment_regressor = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        topic_logits = self.topic_classifier(pooled_output)
        sentiment_score = self.sentiment_regressor(pooled_output)
        return topic_logits, sentiment_score

# --- Training Loop (modified for train/val split) ---
def train_model(model, train_dataloader, val_dataloader, optimizer, topic_loss_fn, sentiment_loss_fn, device, num_epochs, lambda_topic, lambda_sentiment):
    best_val_loss = float('inf') # Initialize
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            topic_labels = batch['topic_label'].to(device)
            sentiment_scores = batch['sentiment_score'].to(device)

            optimizer.zero_grad()
            topic_logits, sentiment_scores_pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            topic_loss = topic_loss_fn(topic_logits, topic_labels)
            sentiment_loss = sentiment_loss_fn(sentiment_scores_pred.squeeze(), sentiment_scores)
            loss = (lambda_topic * topic_loss) + (lambda_sentiment * sentiment_loss)
            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()
        avg_train_loss = total_train_loss / len(train_dataloader)

        # --- Validation ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                topic_labels = batch['topic_label'].to(device)
                sentiment_scores = batch['sentiment_score'].to(device)
                topic_logits, sentiment_scores_pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                topic_loss = topic_loss_fn(topic_logits, topic_labels)
                sentiment_loss = sentiment_loss_fn(sentiment_scores_pred.squeeze(), sentiment_scores)
                loss = (lambda_topic * topic_loss) + (lambda_sentiment * sentiment_loss)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_dataloader)

        print(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Simple checkpointing (save the model if validation loss improves)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "best_model.pth")
            print("    Saved best model!")

    print("Training Complete!")
# --- Evaluation Function (same as before) ---

def evaluate_model(model, dataloader, device):
    model.eval()
    all_predicted_topics = []
    all_true_topics = []
    all_predicted_sentiments = []
    all_true_sentiments = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            topic_logits, sentiment_scores_pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            topic_probs = torch.softmax(topic_logits, dim=-1)
            predicted_topic_labels = torch.argmax(topic_probs, dim=-1)
            all_predicted_topics.extend(predicted_topic_labels.cpu().numpy())
            all_true_topics.extend(batch['topic_label'].cpu().numpy())
            all_predicted_sentiments.extend(sentiment_scores_pred.squeeze().cpu().numpy())
            all_true_sentiments.extend(batch['sentiment_score'].cpu().numpy())

    accuracy = accuracy_score(all_true_topics, all_predicted_topics)
    rmse = np.sqrt(mean_squared_error(all_true_sentiments, all_predicted_sentiments))

    return accuracy, rmse, all_predicted_topics, all_true_topics, all_predicted_sentiments, all_true_sentiments

In [16]:
from torch.utils.data import Dataset, DataLoader, random_split

In [20]:
# --- Main Execution ---
if __name__ == "__main__":
    df = clean
    model_name = 'bert-base-uncased'
    max_length = 64
    batch_size = 16
    learning_rate = 1e-4
    num_epochs = 8
    lambda_topic = 1.0
    lambda_sentiment = 1.0

    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    dataset = SubCommentDatasetMultiTask(df, tokenizer, max_length)

    # --- Train/Val/Test Split ---
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle validation data
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) #no need to shuffle test


    num_labels = dataset.num_labels
    model = BertMultiTask(model_name, num_topics=num_labels)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    topic_loss_fn = nn.CrossEntropyLoss()
    sentiment_loss_fn = nn.MSELoss()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # --- Train the Model ---
    train_model(model, train_dataloader, val_dataloader, optimizer, topic_loss_fn, sentiment_loss_fn,
                device, num_epochs, lambda_topic, lambda_sentiment)

    # --- Evaluate on Train and Test Sets ---
    print("\nEvaluating on Training Set:")
    train_accuracy, train_rmse, _, _, _, _ = evaluate_model(model, train_dataloader, device)
    print(f"  Train Accuracy (Topic): {train_accuracy:.4f}")
    print(f"  Train RMSE (Sentiment): {train_rmse:.4f}")

    print("\nEvaluating on Val Set:")
    val_accuracy, val_rmse, _, _, _, _ = evaluate_model(model, val_dataloader, device)
    print(f"  Val Accuracy (Topic): {val_accuracy:.4f}")
    print(f"  Val RMSE (Sentiment): {val_rmse:.4f}")

    print("\nEvaluating on Test Set:")
    test_accuracy, test_rmse, _, _, _, _ = evaluate_model(model, test_dataloader, device)
    print(f"  Test Accuracy (Topic): {test_accuracy:.4f}")
    print(f"  Test RMSE (Sentiment): {test_rmse:.4f}")


Epoch: 1, Train Loss: 1.0501, Val Loss: 0.7401
    Saved best model!
Epoch: 2, Train Loss: 0.5679, Val Loss: 0.7090
    Saved best model!
Epoch: 3, Train Loss: 0.3841, Val Loss: 0.6469
    Saved best model!
Epoch: 4, Train Loss: 0.2781, Val Loss: 0.6565
Epoch: 5, Train Loss: 0.2297, Val Loss: 0.6429
    Saved best model!
Epoch: 6, Train Loss: 0.2634, Val Loss: 0.6654
Epoch: 7, Train Loss: 0.2022, Val Loss: 0.6670
Epoch: 8, Train Loss: 0.1904, Val Loss: 0.8160
Training Complete!

Evaluating on Training Set:
  Train Accuracy (Topic): 0.9418
  Train RMSE (Sentiment): 0.3078

Evaluating on Val Set:
  Test Accuracy (Topic): 0.8233
  Test RMSE (Sentiment): 0.4550

Evaluating on Test Set:
  Test Accuracy (Topic): 0.8355
  Test RMSE (Sentiment): 0.4567


In [21]:
import os
import torch

def save_model(model, tokenizer, output_dir: str):
    """
    Saves a trained Hugging Face Transformers model and tokenizer.

    Args:
        model: The trained PyTorch model (e.g., BertMultiTask).
        tokenizer: The tokenizer (e.g., BertTokenizerFast).
        output_dir: The directory where you want to save the model and tokenizer.
                     This directory will be created if it doesn't exist.
    """

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    print(f"Saving model and tokenizer to {output_dir}")

    # Save the model's state_dict (recommended for PyTorch models)
    model_to_save = model.module if hasattr(model, "module") else model  # Handle DataParallel/Distributed
    torch.save(model_to_save.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))


    # Save the tokenizer's configuration and vocabulary
    tokenizer.save_pretrained(output_dir)

    print("Model and tokenizer saved successfully!")

# --- Example Usage (after training is complete) ---

# Assuming your trained model is named 'model' and your tokenizer is 'tokenizer'
output_directory = "my_saved_model"  # Choose a directory name
save_model(model, tokenizer, output_directory)

# --- How to download from Colab (add this to a separate Colab cell) ---
# Option 1: Zip the directory and download
import shutil
shutil.make_archive(output_directory, 'zip', output_directory)

from google.colab import files
files.download(f"{output_directory}.zip")

# Option 2: (If using Google Drive) Mount Drive and copy
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r my_saved_model/ "/content/drive/MyDrive/my_models/"  # Adjust path as needed


# --- How to load the saved model (in a new session or script) ---

def load_model(model_class, tokenizer_class, output_dir: str, num_topics: int):
    """
    Loads a trained Hugging Face Transformers model and tokenizer.

    Args:
        model_class:  The *class* of your model (e.g., BertMultiTask).
        tokenizer_class: The *class* of your tokenizer (e.g., BertTokenizerFast).
        output_dir: The directory where the model and tokenizer were saved.
        num_topics: Number of topics (needed to initialize the model).

    Returns:
        The loaded model and tokenizer.
    """
    # Load the tokenizer
    tokenizer = tokenizer_class.from_pretrained(output_dir)

    # Load the model (initialize and then load state_dict)
    model = model_class(base_model_name='bert-base-uncased', num_topics=num_topics)  # Use correct base model name
    model.load_state_dict(torch.load(os.path.join(output_dir, "pytorch_model.bin")))
    model.eval()  # Set to evaluation mode

    return model, tokenizer

# Example of loading:
# from transformers import BertTokenizerFast
# from your_model_file import BertMultiTask  # Assuming your model class is in 'your_model_file.py'
# loaded_model, loaded_tokenizer = load_model(BertMultiTask, BertTokenizerFast, "my_saved_model", num_topics=6) #replace with your number of topics

Saving model and tokenizer to my_saved_model
Model and tokenizer saved successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>